In [2]:
import numpy as np

In [25]:
class Variable:
    def __init__(self, data):
        self.data = data

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output

    def forward(self, input):
        raise NotImplementedError()

class Squre(Function):
    def forward(self, x):
        return x**2

class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [36]:
def numerical_diff(f, x, eps=1e-4):
    return (f(Variable(x.data+eps)).data - f(Variable(x.data-eps)).data) / (2*eps)

In [37]:
f = Squre()
x = Variable(np.array(2.0))
print(numerical_diff(f, x))

4.000000000004
